In [ ]:
import sys
sys.path.append('/home/vincent/git/GPflow')
from matplotlib import pyplot as plt
%matplotlib inline
import GPflow.kernels
from GPflow.likelihoods import Bernoulli, Gaussian
from GPflow.svgp import SVGP
from GPflow.svgp_additive import SVGP_additive2 
import numpy as np   
from scipy.special import erf

In [ ]:
np.random.seed(0)


# build a very simple data set:
N = 3000
D = 2
M = [1,30]
X = np.random.rand(N, D)-.5

#Z needs to be list of (M,1) array
z1 = np.array([[1]])
z2 = np.random.rand(M[1], 1)-.5
Z = [z1,z2]

fl = lambda x : x
fnl = lambda x : 4*x*np.exp(-np.abs(x)*4)


lik = 'Gaussian'
lik = 'Bernoulli'


s_n = 0.1
F = np.zeros((N,1))
Fs = np.zeros((N,D))

Fs[:,0] = fl(X[:,0])
Fs[:,1] = fnl(X[:,1])

F = np.sum(Fs,axis=1).reshape(N,1)

if lik == 'Gaussian':
    Y = F + np.random.randn(N,1) * s_n
elif lik == 'Bernoulli':
    phi =lambda x: 0.5*(1+erf(x/np.sqrt(2)))
    B = phi(F)
    Y = (np.random.rand(N,1)<B).astype(int)

for d in range(D):
    plt.plot(X[:,d],Fs[:,d],'x')
plt.show()

Xg,Yg = np.meshgrid(X[:,0],X[:,1])
plt.scatter(X[:,0],X[:,1],c=B)
plt.show()

plt.hist(F)
plt.show()



plt.hist(Y)
plt.show()


In [ ]:

if lik == 'Gaussian':
    likelihood = Gaussian()
    likelihood.variance = 0.01
elif lik == 'Bernoulli':
    likelihood = Bernoulli()

k1 = GPflow.kernels.Linear(1)
k2 = GPflow.kernels.RBF(1)
ks = [k1,k2]

#Z = [Z[:,d].reshape(M, 1).copy() for d in range(D)]
m = SVGP_additive2(X, Y, ks, likelihood, Z)

m.optimize()
    
Yp, Vp = m.predict_f(X)
Sp = np.sqrt(Vp)

rmse = np.sqrt(np.mean((Yp - Y) ** 2))
print "RMSE: %.3f" % rmse


In [ ]:
col ='rg'
for d in range(D):
    m.set_prediction_subset_ds([d])
    Yd = m.predict_f(X)[0]
    print Yd.shape
    plt.plot(X[:,d],Fs[:,d],'.',c=col[d])
    plt.plot(X[:,d],Yd,'x',c=col[d])
plt.show()